In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
#from sklearn.model_selection import training_test_split
#from sklean
import os 
import pandas as pd
import utils

In [2]:
X, supply, demand = utils.LoadDataset('./training_set/DesignMatrices/')
X.shape, supply.shape, demand.shape

X=X[:, :2]

In [3]:
Xtrain, supplyTrain, demandTrain, Xval, supplyVal, demandVal = utils.SplitByRegion(X, supply, demand, .7)

In [4]:
print 'Xtrain = {}, demandTrain = {}, supplyTrain = {}'.format(Xtrain.shape, demandTrain.shape, supplyTrain.shape)
print 'Xval = {}, demandVal = {}, supplyVal = {}'.format(Xval.shape, demandVal.shape, supplyVal.shape)

Xtrain = (99981L, 2L), demandTrain = (99981L,), supplyTrain = (99981L,)
Xval = (42899L, 2L), demandVal = (42899L,), supplyVal = (42899L,)


In [5]:
def getMAE(truths, preds): 
    return (np.mean(np.abs(truths-preds)))

In [6]:
bestscore = np.inf
bestmodel, bestdeg = None, None
for d in np.arange(1,11):
    Xnew=PolynomialFeatures(degree=d).fit_transform(Xtrain)
    model=linear_model.LinearRegression()
    model.fit(Xnew, demandTrain-supplyTrain)
    
    ypreds=model.predict(PolynomialFeatures(degree=d).fit_transform(Xval))
    print 'degree = {}\tMAE={}'.format(d, getMAE(demandVal-supplyVal, ypreds))
    
    if (bestscore > getMAE(demandVal-supplyVal, ypreds)):
        bestscore = getMAE(demandVal-supplyVal, ypreds)
        bestmodel = model
        bestdeg = d

print bestdeg 

degree = 1	MAE=12.1009173524
degree = 2	MAE=12.0096973902
degree = 3	MAE=12.1286066428
degree = 4	MAE=12.4790431
degree = 5	MAE=12.4640253538
degree = 6	MAE=12.4689530508
degree = 7	MAE=25.8272890344
degree = 8	MAE=13.0127172817
degree = 9	MAE=13.0698694111
degree = 10	MAE=13.1181202985
2


In [40]:
fnames = [x for x in sorted(os.listdir('./test_set/DesignMatrices/')) if x[0]!='.']
for fname in fnames: 
    Xtestpd = pd.read_csv('./test_set/DesignMatrices/'+fname)
    Xtest = Xtestpd[['RegionID', 'Timeslot']].values
    
    Ytest = bestmodel.predict(PolynomialFeatures(degree=bestdeg).fit_transform(Xtest))
    
    myout = pd.DataFrame(Ytest, columns=['gap'])
    myout['district_date_slot'] = Xtestpd['RegionHash']
    #myout['date'] = fname.split('.')[0]
    #myout
    myout['district_date_slot'] = myout.district_date_slot.astype(str).str.cat(Xtestpd.Date.astype(str), sep='_')
    myout['district_date_slot'] = myout.district_date_slot.astype(str).str.cat(Xtestpd.Timeslot.astype(str), sep='-')
    myout=myout.set_index('district_date_slot')
    
    kaggleout = pd.read_csv('sample.csv', index_col='district_date_slot')
    break

In [41]:
kaggleout.head()

,gap
district_date_slot,
90c5a34f06ac86aee0fd70e2adce7d8a_2016-01-23-1,0
f2c8c4bb99e6377d21de71275afd6cd2_2016-01-23-1,0
58c7a4888306d8ff3a641d1c0feccbe3_2016-01-23-1,0
b26a240205c852804ff8758628c0a86a_2016-01-23-1,0
4b9e4cf2fbdc8281b8a1f9f12b80ce4d_2016-01-23-1,0


In [42]:
myout.head()

,gap
district_date_slot,
d4ec2125aff74eded207d2d915ef682f_2016-01-23-45,8.900349
2407d482f0ffa22a947068f2551fe62c_2016-01-23-79,11.505251
b26a240205c852804ff8758628c0a86a_2016-01-23-93,8.866418
4725c39a5e5f4c188d382da3910b3f3f_2016-01-23-103,10.232886
dd8d3b9665536d6e05b29c2648c0e69a_2016-01-23-115,8.145178
